가장 인기 있는 앙상블 방법: 배깅, 부스팅, 스태킹



# 7.1 투표 기반 분류기

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score

x, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=42)

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [2]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.888
SVC 0.896
VotingClassifier 0.904


# 7.2.1 사이킷런의 배깅과 페이스팅

In [3]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,max_samples=100, bootstrap=True, n_jobs=-1
)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

# 7.2.2 oob 평가

- 배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러번 샘플링되고, 어떤 것은 전혀 선택되지 않을 수 있다. 이는 평균적으로 각 예측기에 훈련 샘플의 약 635 정도만 샘플링된다는 것을 의미한다. 선택되지 않은 훈련 샘플의 나머지 37%를 oob샘플이라고 부른다.
- 앙상블의 평가는 각 예측기의 oob평가를 평균하여 얻는다.
- oob_score=True로 지정하면 훈련이 끝난 후 자동으로 oob 평가를 수행한다.


In [4]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True
)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9013333333333333

In [5]:
# 테스트 세트에서의 정확도와 매우 비슷
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.904

In [6]:
# oob 함수의 결정함수 -> 각 훈련샘플의 클래스 확률을 반환함

bag_clf.oob_decision_function_

array([[0.39175258, 0.60824742],
       [0.35028249, 0.64971751],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.08376963, 0.91623037],
       [0.37430168, 0.62569832],
       [0.00512821, 0.99487179],
       [1.        , 0.        ],
       [0.95454545, 0.04545455],
       [0.8       , 0.2       ],
       [0.01117318, 0.98882682],
       [0.80319149, 0.19680851],
       [0.81632653, 0.18367347],
       [0.95348837, 0.04651163],
       [0.06010929, 0.93989071],
       [0.00543478, 0.99456522],
       [0.98170732, 0.01829268],
       [0.94444444, 0.05555556],
       [1.        , 0.        ],
       [0.03910615, 0.96089385],
       [0.33701657, 0.66298343],
       [0.8952381 , 0.1047619 ],
       [1.        , 0.        ],
       [0.96276596, 0.03723404],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.63218391, 0.36781609],
       [0.

# 7.4 랜덤 포레스트

In [7]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [8]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

RandomForestClassifer는 몇가지 예외가 있지만 BaggingClassifer의 매개변수를 모두 가지고 있다.

# 7.4.2 특성중요도

In [9]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])

for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.09918227114416296
sepal width (cm) 0.0229989676501626
petal length (cm) 0.4047158404894371
petal width (cm) 0.47310292071623733
